In [1]:
from __future__ import print_function
import os
from urdf_parser_py.urdf import URDF
from scipy.spatial.transform import Rotation
import time as timer
import numpy as np
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% !important; } </style>"))
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
import rospy
from sensor_msgs.msg import JointState

In [2]:
PROJ_DIR = os.getcwd()
JOINT_NAMES = ["shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"]
LINK_NAMES = ['base_link', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tool0']
ZERO_JOINT_POSE = np.array([0, -np.pi*0.6 , np.pi*0.6,0,0,0])

# Ros setting

In [3]:
def get_publisher():
    pub = rospy.Publisher('/joint_states', JointState, queue_size=10)
    rospy.init_node('joint_state_publisher', anonymous=True)
    rate = rospy.Rate(100) # 10hz
    joints = JointState()
    joints.header.seq += 1
    joints.header.stamp = rospy.Time.now()
    joints.name = ["shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"]
    joints.position = [0, np.deg2rad(-90), 0, np.deg2rad(-90), 0, 0]
    # Publish the marker
    while pub.get_num_connections() < 1:
        print("Please create a subscriber to the marker");
        timer.sleep(1)
    print('publication OK')
    pub.publish(joints);
    print('published: {}'.format(joints.position))
    return pub, joints, rate

def show_motion(pose_list, marker_list, pub, joints):
    for pvec in pose_list:
        joints.header.seq += 1
        joints.header.stamp = rospy.Time.now()
        joints.position = pvec.tolist()
        pub.publish(joints);
        for marker in marker_list:
            marker.move_marker({joints.name[i]: joints.position[i] for i in range(len(joint_names))})
        print('published: {}'.format(joints.position), end="\r")

# joint tools

In [4]:
def get_parent_joint(link_name, urdf_content):
    for joint in urdf_content.joints:
        if joint.child == link_name:
            return joint.name
        
def get_tf(to_link, joint_dict, urdf_content, from_link='base_link'):
    T = np.identity(4)
    T_J = np.identity(4)
    link_cur = to_link
    while link_cur != from_link:
        parent_joint = urdf_content.joint_map[get_parent_joint(link_cur, urdf_content)]
        if parent_joint.type == 'revolute':
            T_J[:3,:3] = Rotation.from_rotvec(np.array(parent_joint.axis)*joint_dict[parent_joint.name]).as_dcm()
        elif parent_joint.type == 'fixed':
            pass
        else:
            raise NotImplementedError  
        Toff = np.identity(4)
        Toff[:3,3] = parent_joint.origin.xyz
        Toff[:3,:3] = Rotation.from_euler("xyz", 
                                          parent_joint.origin.rpy, degrees=False).as_dcm()
        T = np.matmul(Toff, np.matmul(T_J,T))
        link_cur = parent_joint.parent
    return T

def get_transformation(link_name):
    return "T_{link_name}".format(link_name=link_name)

def joint_list2dict(joint_list, joint_names=JOINT_NAMES):
    return {joint_names[i]: joint_list[i] for i in range(len(joint_names))}

# Geometry items

In [5]:
class GeometryItem(object):
    def __init__(self, name, link_name, urdf_content, color=(0,1,0,1), display=True, collision=True):
        self.color = color
        self.display = display
        self.collision = collision
        self.urdf_content = urdf_content
        self.set_name(name)
        self.set_link(link_name)
    
    def get_adjacent_links(self):
        adjacent_links = [self.link_name]
        for k, v in self.urdf_content.joint_map.items():
            if v.parent == self.link_name:
                adjacent_links += [v.child]
            if v.child == self.link_name:
                adjacent_links += [v.parent]
        return list(set(adjacent_links))
    
    def set_name(self, name):
        self.name = name
        
    def set_link(self, link_name):
        self.link_name = link_name
        self.link = self.urdf_content.link_map[link_name]
        self.Tname = get_transformation(self.link_name)
        self.adjacent_links = self.get_adjacent_links()
        
    def get_representation(self):
        raise NotImplementedError
        
    def get_radius(self):
        raise NotImplementedError
        
    def get_scale(self):
        raise NotImplementedError
    
    def get_transformation(self):
        orientation = self.get_orientation()
        angle_option = ""
        if np.sum(np.abs(orientation[:3]))>1e-4:
            xyzrot = Rotation.from_quat(orientation).as_euler("xyz")
            for i in range(len(xyzrot)):
                if abs(xyzrot[i])>1e-4:
                    angle_option = "*rotate_{axis}({val})".format(axis="xyz"[i], val=xyzrot[i])+angle_option
        center = self.get_center()
        center_option = ""
        if np.sum(np.abs(center))>1e-4:
            for i in range(len(center)):
                if abs(center[i])>1e-4:
                    center_option += "*translate_{axis}({val})".format(axis="xyz"[i], val=center[i])
        tf_text = "{Tname}{center_option}{angle_option}".format(
            Tname=self.Tname, center_option=center_option, angle_option=angle_option)
        return tf_text
                
        
    def get_tf(self, joint_dict):
        T = get_tf(to_link=self.link_name, joint_dict=joint_dict, urdf_content=self.urdf_content)
        T = np.matmul(T, self.get_offset_tf())
        return T
        
    def get_offset_tf(self):
        T = np.identity(4)
        T[:3,3] = self.get_center()
        T[:3,:3] = Rotation.from_quat(self.get_orientation()).as_dcm()
        return T
        
    def get_center(self):
        return self.center
        
    def get_orientation(self):
        return self.orientation
        
    def set_center(self, center):
        self.center = center
        
    def set_orientation(self, orientation):
        self.orientation = orientation

        
class GeoSphere(GeometryItem):
    def __init__(self, center, radius, **kwargs):
        self.center, self.radius = center, radius
        self.orientation= (0,0,0,1)
        super(GeoSphere, self).__init__(**kwargs)
        
    def get_representation(self):
        return "MultiSphere({{Vector({},{},{})}},{{ {} }})".format(0, 0, 0, self.radius)
        
    def get_radius(self):
        return 0
        
    def get_scale(self):
        return [self.radius, self.radius, self.radius]

        
class GeoBox(GeometryItem):
    def __init__(self, center, BLH, **kwargs):
        self.center, self.BLH = center, BLH
        self.orientation= (0,0,0,1)
        super(GeoBox, self).__init__(**kwargs)
        
    def get_representation(self):
        return "Box({B},{L},{H})".format(B=self.BLH[0], L=self.BLH[1], H=self.BLH[2])
        
    def get_radius(self):
        return 0
        
    def get_scale(self):
        return self.BLH


class GeoSegment(GeometryItem):
    def __init__(self, point0, axis, length, radius, **kwargs):
        self.point0, self.axis, self.length, self.radius = \
            point0, axis, length, radius
        self.orientation=self.get_axis_quat()
        self.center = list(self.point0)
        self.center["XYZ".index(axis)] += length/2
        super(GeoSegment, self).__init__(**kwargs)
    
    def get_axis_quat(self):
        if self.axis == "X":
            return Rotation.from_rotvec((0,np.pi/2,0)).as_quat()
        elif self.axis == "Y":
            return Rotation.from_rotvec((-np.pi/2,0,0)).as_quat()
        elif self.axis == "Z":
            return Rotation.from_rotvec((0,0,0)).as_quat()
        
    def get_representation(self):
        return "CapsuleZ({radius},{length})".format(radius=self.radius,length=self.length)
        
    def get_radius(self):
        return self.radius
        
    def get_scale(self):
        return (self.radius*2, self.radius*2, self.length)
    
class GeoPointer:
    def __init__(self, _object, direction=None):
        assert direction is not None, "GeoPoint need direction"
        self.__direction = direction
        self.direction = self.__direction
        self.object = _object
    
    def set_direction(self, direction):
        self.direction = direction
    
    def get_direction(self):
        return self.direction

# Constraint

In [6]:
def make_point_pair_constraint(obj1, obj2, varname, constraint_name):
    return """
    local {varname}1 = {repre1}
    local {varname}2 = {repre2}""".format(
        varname=varname, repre1=obj1.get_representation(), repre2=obj2.get_representation()) + \
    """
    dist_{varname} = distance_between({T1},{varname}1,{ctem1radius},margin,{T2},{varname}2,{ctem2radius},margin)
    Constraint{{
        context=ctx,
        name="{constraint_name}",
        expr = dist_{varname},
        priority = 2,
        K        = K
    }}""".format(
        constraint_name=constraint_name,
        T1=obj1.get_transformation(), ctem1radius=0,
        T2=obj2.get_transformation(), ctem2radius=0,
        varname=varname
    )

def make_dir_constraint(pointer1, pointer2, name, constraint_name):
    return """
    vec1 = vector{vec1}
    vec2 = vector{vec2}
    angle_{name} = angle_between_vectors(vec1,rotation(inv({T1})*{T2})*vec2)""".format(
        T1=pointer1.object.get_transformation(),
        T2=pointer2.object.get_transformation(),
        vec1=tuple(pointer1.direction), vec2=tuple(pointer2.direction), name=name) + \
    """
    Constraint{{
        context=ctx,
        name="{constraint_name}",
        expr = angle_{name},
        priority = 2,
        K        = K
    }}""".format(
        constraint_name=constraint_name,
        name=name
    )
    

# define distances
def make_distance_constraint(ctem1, ctem2):
    return """
    Constraint{{
    context=ctx,
        name="{constraint_name}",
        expr = distance_between({T1},{ctem1},{ctem1radius},margin,{T2},{ctem2},{ctem2radius},margin),
        target_lower = 0,
        priority = 0
    }}""".format(
            constraint_name=ctem1.name+"_"+ctem2.name,
            T1=ctem1.get_transformation(), ctem1=ctem1.name, ctem1radius=ctem1.get_radius(),
            T2=ctem2.get_transformation(),ctem2=ctem2.name, ctem2radius=ctem2.get_radius())

def make_directed_point_constraint(etasl_simulator, pointer1, pointer2, name):
    constraint_name_point = "point_pair_{name}".format(name=name)
    constraint_name_dir = "dir_pair_{name}".format(name=name)
    pair_constraint = make_point_pair_constraint(pointer1.object, pointer2.object, constraint_name_point, constraint_name_point)
    dir_constraint = make_dir_constraint(pointer1, pointer2, name=constraint_name_dir, constraint_name=constraint_name_dir)
    etasl_simulator.readTaskSpecificationString(pair_constraint)
    etasl_simulator.readTaskSpecificationString(dir_constraint)
    etasl_simulator.readTaskSpecificationString(
        'ctx:setOutputExpression("dist_{constraint_name_point}",dist_{constraint_name_point})'.format(constraint_name_point=constraint_name_point))
    etasl_simulator.readTaskSpecificationString(
        'ctx:setOutputExpression("angle_{constraint_name_dir}",angle_{constraint_name_dir})'.format(constraint_name_dir=constraint_name_dir))

def update_pointer(action_point):
    Toff = action_point.object.get_offset_tf()
    action_point.point = np.matmul(Toff, list(action_point.point_dir[0])+[1])[:3]
    action_point.direction = np.matmul(Toff[:3,:3], action_point.point_dir[1])
    if hasattr(action_point, "pointer"):
        action_point.pointer.object.set_link(action_point.object.link_name)
        action_point.pointer.object.set_center(action_point.point)
        action_point.pointer.set_direction(action_point.direction)
    else:
        action_point.pointer = GeoPointer(direction=action_point.direction, 
                                  _object=GeoSphere(name=action_point.name_constraint, center=action_point.point, radius=0, 
                                            link_name=action_point.object.link_name, urdf_content=action_point.object.urdf_content)
                                 )

class ActionPoint:
    def add_constraints(self, etasl_simulator):
        raise NotImplementedError

class VacuumPoint(ActionPoint):
    def __init__(self, name, _object, point_dir):
        self.name = name
        self.object = _object
        self.point_dir = point_dir
        self.name_constraint = "vac_{objname}_{name}".format(objname=self.object.name, name=self.name)
        update_pointer(self)
        
    
    def add_constraints(self, etasl_simulator, gripper_pointer):
        update_pointer(self)
        make_directed_point_constraint(etasl_simulator, self.pointer, gripper_pointer, self.name_constraint)
            
class PlacePoint(ActionPoint):
    def __init__(self, name, _object, point_dir):
        self.name = name
        self.object = _object
        self.point_dir = point_dir
        self.name_constraint = "place_{objname}_{name}".format(objname=self.object.name, name=self.name)
        update_pointer(self)
        
    def add_constraints(self, etasl_simulator, place_pointer):
        update_pointer(self)
        make_directed_point_constraint(etasl_simulator, self.pointer, place_pointer, self.name_constraint)
    
class ObjectAction:
    def __init__(self):
        raise NotImplementedError("ObjectAction is abstract class")
        
class BoxGrip(ObjectAction):
    def __init__(self, _object):
        self.object = _object
        Xhalf, Yhalf, Zhalf = np.array(_object.BLH)/2
        self.grip_points_dict = {
            "top": VacuumPoint("top", _object, ([0,0,Zhalf], [0,0,-1])),
            "bottom": VacuumPoint("bottom", _object, ([0,0,-Zhalf], [0,0,1])),
            "right": VacuumPoint("right", _object, ([Xhalf,0,0], [-1,0,0])),
            "left": VacuumPoint("left", _object, ([-Xhalf,0,0], [1,0,0])),
            "front": VacuumPoint("front", _object, ([0,-Yhalf,0], [0,1,0])),
            "back": VacuumPoint("back", _object, ([0,Yhalf,0], [0,-1,0]))
        }
        
    def add_constraints(self, point_name, etasl_simulator, gripper_pointer):
        self.grip_points_dict[point_name].add_constraints(etasl_simulator, gripper_pointer)
        
class BoxPlace(ObjectAction):
    def __init__(self, _object):
        self.object = _object
        Xhalf, Yhalf, Zhalf = np.array(_object.BLH)/2
        self.place_point_dict = {
            "top": PlacePoint("top", _object, ([0,0,Zhalf], [0,0,-1])),
            "bottom": PlacePoint("bottom", _object, ([0,0,-Zhalf], [0,0,1])),
            "right": PlacePoint("right", _object, ([Xhalf,0,0], [-1,0,0])),
            "left": PlacePoint("left", _object, ([-Xhalf,0,0], [1,0,0])),
            "front": PlacePoint("front", _object, ([0,-Yhalf,0], [0,1,0])),
            "back": PlacePoint("back", _object, ([0,Yhalf,0], [0,-1,0]))
        }
        
    def add_constraints(self, point_name, etasl_simulator, place_pointer):
        self.place_point_dict[point_name].add_constraints(etasl_simulator, place_pointer)

def add_joint_constraints(etasl, joint_names=JOINT_NAMES):
    for i in range(len(joint_names)):
        etasl.readTaskSpecificationString("""
            target_{joint_name} = ctx:createInputChannelScalar("target_{joint_name}",0.0)
            Constraint{{
                context=ctx,
                name="constraint_{joint_name}",
                expr=robot_jval[{index}]-target_{joint_name},
                priority    = 2,
                K           = K
            }}
            """.format(joint_name=joint_names[i], index=i+1))

# Visualization class

In [7]:
import rospy
from visualization_msgs.msg import Marker
import uuid

class GeoMarker:
    ID_COUNT = 0
    def __init__(self, geometry, urdf_content, mark_name='visualization_marker', node_name=None):
        if node_name is None:
            node_name = mark_name + "_pub"
        self.geometry = geometry
        self.urdf_content = urdf_content
        self.pub = rospy.Publisher(mark_name, Marker, queue_size=10)
#         rospy.init_node(node_name, anonymous=True)
        rate = rospy.Rate(1) # 10hz
        # Publish the marker
        while self.pub.get_num_connections() < 1:
            print("Please create a subscriber to the marker");
            timer.sleep(1)
        print('publication OK')
    
    def set_marker(self, joint_dict):
        GeoMarker.ID_COUNT += 1
        self.marker = Marker()
        self.marker.header.frame_id = "/base_link"
        self.marker.header.stamp = rospy.Time.now()
        self.marker.ns = "basic_shapes"
        self.marker.id = GeoMarker.ID_COUNT
        self.marker.type = self.get_type()
        self.marker.action = Marker.ADD
        self.marker.scale.x, self.marker.scale.y, self.marker.scale.z = self.geometry.get_scale()
        self.marker.color.r, self.marker.color.g, self.marker.color.b, self.marker.color.a  = self.geometry.color
        self.marker.lifetime = rospy.Duration()
        self.__set_position(joint_dict)
        self.pub.publish(self.marker);
        
    def __set_position(self, joint_dict):
        T = self.geometry.get_tf(joint_dict)
#         R_off = 
        self.marker.pose.orientation.x, self.marker.pose.orientation.y, \
            self.marker.pose.orientation.z, self.marker.pose.orientation.w = \
            Rotation.from_dcm(T[:3,:3]).as_quat()
        self.marker.pose.position.x, self.marker.pose.position.y, self.marker.pose.position.z = \
            T[:3,3]
        
    def move_marker(self, joint_dict = [0]*6):
        self.marker.action = Marker.MODIFY
        self.marker.header.stamp = rospy.Time.now()
        self.__set_position(joint_dict)
        self.pub.publish(self.marker);
        
    def get_type(self):
        if isinstance(self.geometry, GeoBox):
            return Marker.CUBE
        elif isinstance(self.geometry, GeoSegment):
            return Marker.CYLINDER
        elif isinstance(self.geometry, GeoSphere):
            return Marker.SPHERE
        
    def delete(self):
        self.marker.action = Marker.DELETE
        self.pub.publish(self.marker);
        
    def __del__(self):
        self.marker.action = Marker.DELETE
        self.pub.publish(self.marker);
        
def set_markers(collision_items, joints, joint_names, link_names, urdf_content):
    marker_list = []
    joint_dict = {joints.name[i]: joints.position[i] for i in range(len(joint_names))}
    for link_name in link_names:
        ctems = collision_items[link_name]
        for ctem in ctems:
            if ctem.display:
                marker_list += [GeoMarker(geometry=ctem,urdf_content=urdf_content)]
                marker_list[-1].set_marker(joint_dict)
    return marker_list

# Prepare eTaSL

In [8]:
def get_joint_names_csv(joint_names=JOINT_NAMES):
    jnames_format = ""
    for jname in joint_names:
        if jname not in urdf_content.joint_map \
                or urdf_content.joint_map[jname].axis is  None:
            raise("invalid joint name is defined")
        jnames_format = jnames_format+'"%s",'%jname
    jnames_format = jnames_format[:-1]
    return jnames_format

In [9]:
def get_simulation(collision_items, joint_names = JOINT_NAMES, link_names = LINK_NAMES):
    etasl = etasl_simulator(nWSR=100, cputime=1000, regularization_factor= 1e-6)
    jnames_format = get_joint_names_csv(joint_names)
    # define margin and translation
    transform_text = \
    """
        margin=0.0001
        radius=0.0
    """
    Texpression_text = ""
    ctem_list = []
    for c_key in link_names:
        ctems = collision_items[c_key]
        transform_text += '    u:addTransform("{T_link_name}","{link_name}","base_link")\n'.format(T_link_name=get_transformation(c_key), link_name=c_key)
        Texpression_text += '    {T_link_name} = r.{T_link_name}\n'.format(T_link_name=get_transformation(c_key))
        for i in range(len(ctems)):
            ctem = ctems[i]
            if ctem.collision:
                ctem_list += [ctem]
    # print(transform_text)
    # print(Texpression_text)


    definition_text = """
        require("context")
        require("geometric")
        --require("libexpressiongraph_collision")
        require("collision")
        local u=UrdfExpr();
        local fn = "%s"
        u:readFromFile(fn)
        u:addTransform("ee","tool0","base_link")
    %s
        local r = u:getExpressions(ctx)
        ee = r.ee
    %s
        robot_jname={%s}
        robot_jval = {}
        for i=1,#robot_jname do
           robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
        end

        K=40
    """%(urdf_path, transform_text, Texpression_text, jnames_format)

    # define items
    item_text = ""
    for c_key in link_names:
        ctems = collision_items[c_key]
        for i in range(len(ctems)):
            ctem = ctems[i]
            item_text += """    {name}={ctem}\n""".format(name=ctem.name, ctem=ctem.get_representation())
    # print(item_text)

    constraint_text = ""
    for ctem1 in ctem_list:
        for ctem2 in ctem_list[ctem_list.index(ctem1)+1:]:
            if ctem2.link_name in ctem1.adjacent_links or ctem1.link_name in ctem2.adjacent_links:
                pass
            else:
                constraint_text += make_distance_constraint(ctem1, ctem2)

    etasl.readTaskSpecificationString(definition_text)
    etasl.readTaskSpecificationString(item_text)
    etasl.readTaskSpecificationString(constraint_text)
    
    return etasl

def simulate(etasl, N=400, dt=0.005, joint_names = JOINT_NAMES, 
             initial_jpos = ZERO_JOINT_POSE, inp_lbl=[], inp=[]):
    time = np.arange(0,N)*dt
    pos_lbl = joint_names
    # inp_lbl=['tgt_x','tgt_y','tgt_z']
    # inp=[0.5, 0.0, 0.0]
    
    etasl.setInputTable(inp_lbl,inp)
    etasl.initialize(initial_jpos, pos_lbl)
    etasl.simulate(N=N,dt=dt)

# Task constraints

In [10]:
class ActionTool:
    def __init__(self):
        raise NotImplementedError("ActionTool is abstract class")
        
class VacuumTool(ActionTool):
    def __init__(self, point0, direction, link_name, urdf_content):
        self.point0 = point0
        self.direction = direction
        self.link_name = link_name
        self.name_constraint = "gripper_{link_name}".format(link_name=link_name)
        self.pointer = GeoPointer(direction=direction, 
                                  _object=GeoSphere(name=self.name_constraint, center=point0, radius=0, 
                                            link_name=link_name, urdf_content=urdf_content)
                                 )
        
    def add_constraint(self, action_obj, point_name, etasl_simulator):
        action_obj.add_constraints(point_name, etasl_simulator, self.pointer)
        
    def activate(self, action_obj, collision_items, joint_dict_last):
        Tbo = action_obj.object.get_tf(joint_dict_last)
        Tbt = get_tf(self.link_name, joint_dict_last, urdf_content)
        Tto = np.matmul(np.linalg.inv(Tbt), Tbo)
        action_obj.object.set_center(Tto[:3,3])
        action_obj.object.set_orientation(Rotation.from_dcm(Tto[:3,:3]).as_quat())
        collision_items[action_obj.object.link_name].remove(action_obj.object)
        action_obj.object.set_link(self.link_name)
        collision_items[action_obj.object.link_name] += [action_obj.object]
        
    def deactivate(self, action_obj, collision_items, joint_dict_last, to_link='base_link'):
        assert action_obj.object.link_name == self.link_name, "tool is not activated"
        Tbo = action_obj.object.get_tf(joint_dict_last)
        Tbt = get_tf(to_link, joint_dict_last, action_obj.object.urdf_content)
        Tto = np.matmul(np.linalg.inv(Tbt), Tbo)
        action_obj.object.set_center(Tto[:3,3])
        action_obj.object.set_orientation(Rotation.from_dcm(Tto[:3,:3]).as_quat())
        collision_items[action_obj.object.link_name].remove(action_obj.object)
        action_obj.object.set_link(to_link)
        collision_items[to_link] += [action_obj.object]
    

# Set Main Parameters

In [11]:
urdf_path = os.path.join(PROJ_DIR, "robots", "ur10_robot.urdf")
urdf_content = URDF.from_xml_file(urdf_path)
joint_names = JOINT_NAMES
link_names = LINK_NAMES
pos_last = ZERO_JOINT_POSE

In [12]:
# prepare ros
pub, joints, rate = get_publisher()

Please create a subscriber to the marker
publication OK
published: [0, -1.5707963267948966, 0, -1.5707963267948966, 0, 0]


In [13]:
# prepare joints
collision_items = {"base_link":[GeoSegment((0,0,0), 'Z', 0.1273, 0.08, 
                                           name="base_capsule", link_name="base_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "shoulder_link":[GeoSegment((0,0,0), 'Y', 0.22094, 0.08, 
                             name="shoulder_capsule", link_name="shoulder_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "upper_arm_link":[GeoSegment((0,-0.05,0.0), 'Z', 0.612, 0.06, 
                              name="upper_arm_capsule", link_name="upper_arm_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "forearm_link":[GeoSegment((0,-0,0), 'Z', 0.5723, 0.05, 
                            name="forearm_capsule", link_name="forearm_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "wrist_1_link":[GeoSegment((0,0,0), 'Y', 0.1149, 0.047, 
                            name="wrist_1_capsule", link_name="wrist_1_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "wrist_2_link":[GeoSegment((0,0,0), 'Z', 0.1157, 0.046, 
                            name="wrist_2_capsule", link_name="wrist_2_link", urdf_content=urdf_content, color=(0,1,0,0.5))],
 "wrist_3_link":[GeoSegment((0,0,0), 'Y', 0.0922-0.0522, 0.046, 
                            name="wrist_3_capsule", link_name="wrist_3_link",urdf_content=urdf_content, color=(0,1,0,0.5))],
 "tool0":[]
}
gbox = GeoBox((0.5,-0.2,0.050), (0.1,0.1,0.1), 
              name="box_1", link_name="base_link", urdf_content=urdf_content, color=(0.8,0.8,0.8,1))
gbox_floor = GeoBox((0,0,-0.005), (3,3,0.01), 
                    name="floor", link_name="base_link", urdf_content=urdf_content, color=(0.6,0.6,0.6,1))
gbox_wall = GeoBox((0.7,0.0,0.2), (0.8,0.05,0.4), 
                      name="wall", link_name="base_link", urdf_content=urdf_content, color=(0.4,0.3,0.0,1))
gbox_stepper = GeoBox((0.4,0.4,0.15), (0.3,0.3,0.3), 
                      name="stepper", link_name="base_link", urdf_content=urdf_content, color=(0.4,0.3,0.0,1))
gbox_goal = GeoBox((0.4,0.4,0.3), (0.2,0.2,1e-3), 
                   name="platform", link_name="base_link", urdf_content=urdf_content, color=(0.8,0.0,0.0,1),
                  collision=False)
collision_items["base_link"] += [gbox]
collision_items["base_link"] += [gbox_floor]
collision_items["base_link"] += [gbox_wall]
collision_items["base_link"] += [gbox_stepper]
collision_items["base_link"] += [gbox_goal]
box_grip = BoxGrip(gbox)
box_place = BoxPlace(gbox)
vtool = VacuumTool([0,0,0], [0,0,1],"tool0", urdf_content)

In [14]:
# prepare visualization markers
marker_list = set_markers(collision_items, joints, joint_names, link_names, urdf_content)

Please create a subscriber to the marker
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK
publication OK


# get simulation

In [15]:
t1 = timer.time()
e = get_simulation(collision_items = collision_items, joint_names = JOINT_NAMES, link_names = LINK_NAMES)
t2 = timer.time()
print("sim time: {} ms".format((t2-t1)*1000))

sim time: 142.394065857 ms


# Set grasp slack constraints

In [16]:
vtool.add_constraint(box_grip, "top", e)

# simulate

In [17]:
t1 = timer.time()
# simulate(e, initial_jpos=ZERO_JOINT_POSE)
simulate(e, initial_jpos=pos_last)
t2 = timer.time()
pos_last = e.POS[-1]
joint_dict_last = joint_list2dict(pos_last, joint_names)
print(e.etasl.getOutput())
print("sim time: {} ms".format((t2-t1)*1000))

{'global.dist_point_pair_vac_box_1_top': 5.087596364319325e-07, 'global.angle_dir_pair_vac_box_1_top': 1.4586402587314075e-06}
sim time: 252.997875214 ms


# show_motion

In [18]:
show_motion(e.POS, marker_list, pub, joints)

# activate grasp

In [19]:
vtool.activate(action_obj=box_grip, collision_items=collision_items, joint_dict_last=joint_dict_last)

In [20]:
e = get_simulation(collision_items = collision_items, joint_names = JOINT_NAMES, link_names = LINK_NAMES)

In [21]:
add_joint_constraints(e, joint_names=joint_names)

In [22]:
simulate(e, initial_jpos=pos_last, inp_lbl=['target_%s'%jname for jname in joint_names], inp=list(ZERO_JOINT_POSE))
pos_last = e.POS[-1]
joint_dict_last = joint_list2dict(pos_last, joint_names)
show_motion(e.POS, marker_list, pub, joints)

# get simulation

In [23]:
t1 = timer.time()
e = get_simulation(collision_items = collision_items, joint_names = JOINT_NAMES, link_names = LINK_NAMES)
t2 = timer.time()
print("sim time: {} ms".format((t2-t1)*1000))

sim time: 83.2221508026 ms


# Set place slack constraints

In [24]:
# pointer_plane = GeoPointer(direction=[0,0,1], 
#                            _object = GeoSphere(name="plane_place", center=[0.4,0.2,0], 
#                                                radius=0, link_name="base_link", urdf_content=urdf_content)
#                           )
pointer_plane = GeoPointer(direction=[0,0,1], _object = gbox_goal)
box_place.add_constraints("bottom", e, pointer_plane)

# simulate

In [25]:
simulate(e, N=400,initial_jpos=ZERO_JOINT_POSE)
pos_last = e.POS[-1]
joint_dict_last = joint_list2dict(pos_last, joint_names)
print(e.etasl.getOutput())
show_motion(e.POS, marker_list, pub, joints)

{'global.angle_dir_pair_place_box_1_bottom': 6.5950728116748256e-06, 'global.dist_point_pair_place_box_1_bottom': 4.666873064707033e-08}


# deactivate grasp

In [26]:
vtool.deactivate(action_obj=box_grip, collision_items=collision_items, joint_dict_last=joint_dict_last, to_link='base_link')

In [27]:
e = get_simulation(collision_items = collision_items, joint_names = JOINT_NAMES, link_names = LINK_NAMES)

In [28]:
add_joint_constraints(e, joint_names=joint_names)

In [29]:
simulate(e, initial_jpos=pos_last, inp_lbl=['target_%s'%jname for jname in joint_names], inp=list(ZERO_JOINT_POSE))
pos_last = e.POS[-1]
joint_dict_last = joint_list2dict(pos_last, joint_names)
show_motion(e.POS, marker_list, pub, joints)

In [30]:
e.displayContext()

# delete all

In [31]:
for mkr in marker_list:
    mkr.delete()

# Plotting (using the Bokeh library to provide interactive plots)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv

output_notebook()
print("positions : ")
plotv(e.TIME,e.POS,e.POS_LBL)
print("velocities : ")
plotv(e.TIME,e.VEL,e.POS_LBL)
print("outputs : ")
plotv(e.TIME,e.OUTP,e.OUTP_LBL)